In [ ]:
import market
import metrics

import pandas
import numpy
import itertools
import statsmodels.api as sm

https://earlyretirementnow.com/2017/05/24/the-ultimate-guide-to-safe-withdrawal-rates-part-15-sequence-of-return-risk-part2/

In [ ]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

In [ ]:
m = market.Returns_US_1871()
years = 30
first_year = m.start_year
last_year = 2018 - years + 1

swrs = pandas.DataFrame(index=numpy.arange(first_year, last_year+1), columns=['SWR'])
for start in range(first_year, last_year + 1):
    returns = []
    
    for one_year in zip(range(years), m.iter_from(start)):
        annual_returns = one_year[1]
        real_returns = ((1 + annual_returns.stocks) / (1 + annual_returns.inflation)) - 1
        returns.append(real_returns)        

    swr = float(metrics.pwa(1, 1, returns))
    swrs.loc[start] = swr
swrs.head()

In [ ]:
def get_averages(slice_by=5):
    m = market.Returns_US_1871()
    years = 30
    first_year = m.start_year
    last_year = 2018 - years + 1
    num_slices = years // slice_by
    
    return_columns = ['Return%s' % i for i in range(num_slices)]
    inflation_columns = ['Inflation%s' % i for i in range(num_slices)]

    series = pandas.DataFrame(index=numpy.arange(first_year, last_year+1), columns=return_columns + inflation_columns)
    for start in range(first_year, last_year + 1):
        annual = zip(range(years), m.iter_from(start))
        annual = [x[1] for x in annual]

        returns = [x.stocks for x in annual]
        inflation = [x.inflation for x in annual]

        average_inflation = [metrics.average(x) for x in grouper(inflation, slice_by)]
        average_return = [metrics.average(x) for x in grouper(returns, slice_by)]
        row = series.loc[start]
        
        for i in range(num_slices):
            row['Return%s' % i] = average_return[i]
            row['Inflation%s' %i] = average_inflation[i]
    return series
        
averages = get_averages(slice_by=2)
averages.head()

In [ ]:
Y = swrs.astype(float)
#return_columns = ['Return%s' % i for i in range(4)]
inflation_columns = ['Inflation%s' % i for i in range(14)]
X = averages[inflation_columns].astype(float)
#X = averages.astype(float)
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 5})
predictions = model.predict(X)
model.summary()

In [ ]:
model.params[["Inflation0", "Inflation1", "Inflation2" "Inflation3", "Inflation4", "Inflation5"]].sum()